# Scheduling

In this notebook, we explore how to solve a resource constrained project scheduling problem (RCPSP).

The problem is made of $M$ activities that have precedence constraints. That means that if activity $j\in[1,M]$ is a successor of activity $i\in[1,M]$, then activity $i$ must be completed before activity $j$ can be started

On top of these constraints, each project is assigned a set of K renewable resources where each resource $k$ is available in $R_{k}$ units for the entire duration of the project. Each activity may require one or more of these resources to be completed. While scheduling the activities, the daily resource usage for resource $k$ can not exceed $R_{k}$ units.

Each activity $j$ takes $d_{j}$ time units to complete.

The overall goal of the problem is usually to minimize the makespan,

(Variants: mix of renewable and non-renewable resources / multi-mode)


## Problem definition

Let start with a simple problem which has 5 tasks to execute:

Task 0 has a duration of 1 hour

Task 1 depends on 0 and has a duration of 3 hours

Task 2 depends on 0 and has a duration of 1 hours

Task 3 depends on 2 and has a duration of 3 hours

Task 4 depends on 1 & 3 and has a duration of 2 hours

## Domain construction

In [ ]:
from typing import Any, Dict, List, Optional, Set, Union

from skdecide.builders.domain.scheduling.scheduling_domains import SingleModeRCPSP, SchedulingObjectiveEnum
from skdecide import Distribution, DiscreteDistribution

from skdecide.builders.domain.scheduling.modes import ModeConsumption, ConstantModeConsumption
from skdecide import rollout_episode

class SimpleRCPSPDomain(SingleModeRCPSP):

    def _get_objectives(self) -> List[SchedulingObjectiveEnum]:
        return [SchedulingObjectiveEnum.MAKESPAN]

    def __init__(self):
        self.initialize_domain()

    def _get_max_horizon(self) -> int:
        return 50

    def _get_successors(self) -> Dict[int, List[int]]:
        return {1: [2, 3], 2: [5], 3: [4], 4: [5], 5: []}

    def _get_tasks_ids(self) -> Union[Set[int], Dict[int, Any], List[int]]:
        return {1, 2, 3, 4, 5}

    def _get_tasks_mode(self) -> Dict[int, ModeConsumption]:
        return {
            1: ConstantModeConsumption({'r1': 1,}),
            2: ConstantModeConsumption({'r1': 1,}),
            3: ConstantModeConsumption({'r1': 1,}),
            4: ConstantModeConsumption({'r1': 1,}),
            5: ConstantModeConsumption({'r1': 1,})
        }

    def _get_resource_types_names(self) -> List[str]:
        return ['r1']

    def _get_task_duration(self, task: int, mode: Optional[int] = 1, progress_from: Optional[float] = 0.) -> int:
        all_durations = {1: 1, 2: 3, 3: 1, 4: 3, 5: 2}
        return all_durations[task]

    def _get_original_quantity_resource(self, resource: str, **kwargs) -> int:
        all_resource_quantities = {'r1': 10_000}
        return all_resource_quantities[resource]


In [ ]:
domain = SimpleRCPSPDomain()
domain.set_inplace_environment(False)
state = domain.get_initial_state()

## Let's analyze the graph 

In [ ]:
graph = domain.compute_graph()

### Show the graph

In [ ]:
def to_json(graph):
    pass

# plot graph in javascript

## Let's use the LazyAstar to solve this problem

In [ ]:
from skdecide.hub.solver.lazy_astar import LazyAstar

solver = LazyAstar(from_state=state, heuristic=None)
solver.solve(domain_factory=lambda: domain)